In [ ]:
# imports

import os
import sys
import json
import importlib
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../../'))
sys.path.append(topdir)

import tools.iotools as iotools
import tools.dftools as dftools
import tools.omstools as omstools
import plotting.plottools as plottools

In [ ]:
# load occupancy from dqmio files

mes = ({
    'PXLayer_1': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_1',
    'PXLayer_2': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_2',
    'PXLayer_3': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_3',
    'PXLayer_4': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_4',
})

eras = [
  #'Run2024A-v1', # only commissioning, no lumisections with physics flag set to True
  'Run2024B-v1',
  'Run2024C-v1',
  'Run2024D-v1',
  'Run2024E-v1',
  'Run2024E-v2',
  'Run2024F-v1',
  'Run2024G-v1',
  'Run2024H-v1',
  'Run2024I-v1',
  'Run2024I-v2',
  #'Run2024J-v1'  # pp reference run for heavy ion run; lower pileup and occupancy
]

datadir = '/eos/user/l/llambrec/dialstools-output'
year = '2024'
dataset = 'ZeroBias'
reco = 'PromptReco'

occupancy_info = {}
for era in eras:
    occupancy_info[era] = {}
    mainera, version = era.split('-')
    for melabel, mename in mes.items():
        f = f'{dataset}-{mainera}-{reco}-{version}-DQMIO-{mename}.parquet'
        f = os.path.join(datadir, f)
        df = iotools.read_parquet(f, columns=['run_number', 'ls_number', 'entries'])
        runs = df['run_number'].values
        lumis = df['ls_number'].values
        entries = df['entries'].values
        occupancy_info[era][melabel] = {'runs': runs, 'lumis': lumis, 'entries': entries}

**Part 1: make norm**

In [ ]:
# load trigger rates from json file

hltrate_info = {}
for era in eras:
    hltfile = '../omsdata/hltrate_{}.json'.format(era)
    with open(hltfile, 'r') as f:
        hltrate_info[era] = json.load(f)

In [ ]:
# load oms json

oms_info = {}
for era in eras:
    omsfile = '../omsdata/omsdata_{}.json'.format(era)
    with open(omsfile, 'r') as f:
        oms_info[era] = json.load(f)

In [ ]:
# define and fit normalization function

from scipy.optimize import curve_fit

dosave = True
outputdir = 'normdata'
if not os.path.exists(outputdir): os.makedirs(outputdir)
    

def normfunc(points, a):
    x, y = points
    prefactor = 1e7
    # calculate the product of x and y and do standard scaling
    product = np.multiply(x, y)
    maxproduct = np.amax(product)
    product /= maxproduct
    # settings
    scale = 0.01
    offset = 0.01
    z = prefactor * a * (product + (offset*scale)/(scale+product))
    return z


# loop over mes and eras
norm_info = {}
for era in eras:
    norm_info[era] = {}
    for melabel in mes.keys():
        print(f'Running on era {era}, ME {melabel}...')
        
        # get the data
        runs = occupancy_info[era][melabel]['runs']
        lumis = occupancy_info[era][melabel]['lumis']
        entries = occupancy_info[era][melabel]['entries']
        print('  - Found {} lumisections'.format(len(lumis)))
        runs_filtered = runs
        lumis_filtered = lumis
        entries_filtered = entries
        
        # perform some filtering by physics flag
        physics_flag_mask = omstools.find_oms_attr_for_lumisections(runs_filtered, lumis_filtered, oms_info[era], 'physics_flag', verbose=False).astype(bool)
        print('  - Physics flag filter: {} / {} lumisections passing'.format(np.sum(physics_flag_mask), len(physics_flag_mask)))
        runs_filtered = runs_filtered[physics_flag_mask]
        lumis_filtered = lumis_filtered[physics_flag_mask]
        entries_filtered = entries_filtered[physics_flag_mask]
        
        # remove zero entries
        zero_entries_mask = (entries_filtered > 0).astype(bool)
        print('  - Zero number of entries filter: {} / {} lumisections passing'.format(np.sum(zero_entries_mask), len(zero_entries_mask)))
        runs_filtered = runs_filtered[zero_entries_mask]
        lumis_filtered = lumis_filtered[zero_entries_mask]
        entries_filtered = entries_filtered[zero_entries_mask]
        
        # get trigger rate and pileup for filtered lumisections
        hltrate_filtered = omstools.find_hlt_rate_for_lumisections(runs_filtered, lumis_filtered, hltrate_info[era], 'HLT_ZeroBias_v*', verbose=False)
        pileup_filtered = omstools.find_oms_attr_for_lumisections(runs_filtered, lumis_filtered, oms_info[era], 'pileup', verbose=False)
        
        # do the fit
        fitresult = curve_fit(normfunc, (hltrate_filtered, pileup_filtered), entries_filtered)
        bestcoeffs = fitresult[0]
        print(f'  - Best coefficients: {bestcoeffs}')
        
        # make prediction (on all lumisections, not just filtered ones)
        hltrate = omstools.find_hlt_rate_for_lumisections(runs, lumis, hltrate_info[era], 'HLT_ZeroBias_v*', verbose=False)
        pileup = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'pileup', verbose=False)
        norm = normfunc((hltrate, pileup), *bestcoeffs)
        print(f'  - Calculated norm for {len(norm)} lumisections.')
        norm_info[era][melabel] = {
            'run_number': [int(run) for run in runs],
            'lumisection_number': [int(lumi) for lumi in lumis],
            'norm': list(norm)
        }
        
        # write to file
        if dosave:
            outputfile = f'normdata_{era}_{melabel}.json'
            outputfile = os.path.join(outputdir, outputfile)
            with open(outputfile, 'w') as f:
                json.dump(norm_info[era][melabel], f)

In [ ]:
# investigate particular cases

importlib.reload(omstools)

mekey = 'PXLayer_1'
era = 'Run2024B-v1'
run = 378981
ls = 667

# get OMS info
pileup = omstools.find_oms_attr_for_lumisections([run], [ls], oms_info[era], 'pileup', verbose=True)
print(pileup)

# get trigger rate
hltrate = omstools.find_hlt_rate_for_lumisections([run], [ls], hltrate_info[era], 'HLT_ZeroBias_v*', verbose=True)
print(hltrate)

# get norm
norm = omstools.find_oms_attr_for_lumisections([run], [ls], norm_info[era][mekey], 'norm', verbose=True)
print(norm)

**Part 2: plot result**

In [ ]:
# load norm json

norm_info = {}
for era in eras:
    norm_info[era] = {}
    for melabel in mes.keys():
        normfile = f'normdata/normdata_{era}_{melabel}.json'
        with open(normfile, 'r') as f:
            norm_info[era][melabel] = json.load(f)

In [ ]:
# load oms json (needed for physics flag filter)

oms_info = {}
for era in eras:
    omsfile = '../omsdata/omsdata_{}.json'.format(era)
    with open(omsfile, 'r') as f:
        oms_info[era] = json.load(f)

In [ ]:
def plot_occupancy_vs_norm(
    occupancy_info, norm_info, norm_attr, mes, eras,
    oms_info=None, physics_flag_filter = False,
    zero_entries_filter = False,
    min_entries_filter = None,
    colors='single', xaxlabel='auto', yaxlabel='Occupancy',
    ymax=None, normalize = False,
    dolegend = False):
    # make plot of occupancy vs custom normalization
    
    # define colors
    if colors=='single':
        # use same color for all eras
        # (typically used if plotting only one era)
        colors = ['b']*len(eras)
    if colors=='perera':
        # use a colormap with different colors for different eras
        # (typically used if plotting multiple eras)
        cmap = mpl.colormaps['viridis']
        colors = [cmap(i) for i in np.linspace(0., 1., num=len(eras), endpoint=True)]
    
    # initialize figure
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(18,6), squeeze=False)
    axs = axs.flatten()
    
    # loop over monitoring elements and eras
    for meidx, melabel in enumerate(mes.keys()):
        ax = axs[meidx]
        for eraidx, era in enumerate(eras):
            print('Retrieving data for {}, {}...'.format(melabel, era))
            
            # get occupancy
            runs = occupancy_info[era][melabel]['runs']
            lumis = occupancy_info[era][melabel]['lumis']
            entries = occupancy_info[era][melabel]['entries']
            print('  - Found {} lumisections'.format(len(lumis)))
            
            # perform filtering
            if physics_flag_filter:
                if oms_info is None:
                    raise Exception('Must provide OMS info if physics_flag_filter is set to True.')
                physics_flag_mask = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'physics_flag').astype(bool)
                print('  - Physics flag filter: {} / {} lumisections passing'.format(np.sum(physics_flag_mask), len(physics_flag_mask)))
                runs = runs[physics_flag_mask]
                lumis = lumis[physics_flag_mask]
                entries = entries[physics_flag_mask]
            if zero_entries_filter:
                zero_entries_mask = (entries > 0).astype(bool)
                print('  - Zero number of entries filter: {} / {} lumisections passing'.format(np.sum(zero_entries_mask), len(zero_entries_mask)))
                runs = runs[zero_entries_mask]
                lumis = lumis[zero_entries_mask]
                entries = entries[zero_entries_mask]
            if min_entries_filter is not None:
                min_entries_mask = (entries > min_entries_filter).astype(bool)
                print('  - Min. number of entries filter: {} / {} lumisections passing'.format(np.sum(min_entries_mask), len(min_entries_mask)))
                runs = runs[min_entries_mask]
                lumis = lumis[min_entries_mask]
                entries = entries[min_entries_mask]
            
            # get norm
            values = omstools.find_oms_attr_for_lumisections(runs, lumis, norm_info[era][melabel], norm_attr)
            
            # divide occupancy by trigger rate if requested
            if normalize: entries = np.divide(entries, np.where(values>0, values, 1))
                
            # make the scatter plot
            ax.scatter(values, entries, s=1, alpha=0.25, color=colors[eraidx], label=era)
            if len(eras)==1: ax.text(0.05, 0.95, era + ', ' + melabel, va='top', transform=ax.transAxes, fontsize=12)
        
        # plot aesthetics
        if xaxlabel=='auto': xaxlabel = omsattr
        ax.set_xlabel(xaxlabel, fontsize=15)
        ax.set_ylabel(yaxlabel, fontsize=15)
        if ymax is not None: ax.set_ylim((0, ymax))
        if dolegend:
            ncols = len(eras)%4
            leg = ax.legend(loc='upper left', ncols=ncols)
            for handle in leg.legend_handles:
                handle._sizes = [30]
                handle.set_alpha(1)
        fig.subplots_adjust(bottom=-0.2, left=-0.2)
    return fig, axs

In [ ]:
# make plots

# per era
for era in eras[:1]:
    fig, axs = plot_occupancy_vs_norm(
        occupancy_info, norm_info, 'norm', mes, [era],
        oms_info=oms_info,
        physics_flag_filter=True,
        zero_entries_filter=True,
        #min_entries_filter=0.02e7,
        xaxlabel='Norm', yaxlabel='Occupancy',
    )
    
# eras together
fig, axs = plot_occupancy_vs_norm(
    occupancy_info, norm_info, 'norm', mes, eras,
    oms_info=oms_info,
    physics_flag_filter=True,
    zero_entries_filter=True,
    #min_entries_filter=0.02e7,
    colors='perera', xaxlabel='Norm', yaxlabel='Occupancy',
    ymax=1e7, dolegend=True,
)
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, 'Run 2024, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# make normalized plots

# normalized
fig, axs = plot_occupancy_vs_norm(
    occupancy_info, norm_info, 'norm', mes, eras,
    oms_info=oms_info,
    physics_flag_filter=True,
    zero_entries_filter=True,
    #min_entries_filter=0.02e7,
    normalize=True,
    colors='perera', xaxlabel='Norm', yaxlabel='Occupancy',
    ymax = 3, dolegend=True,
)
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, 'Run 2024, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)